In [1]:
import requests
import pandas as pd
import re
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine
import time
import geopandas as gpd
from geoalchemy2 import Geometry, Geography
import geocoder


engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)
sql_alc_cnxn = engine.connect()

In [2]:
locations = ['Central Business District',
             'Holland Village',
             'Dempsey Hill',
             'Tiong Bahru',
             'Rochester Park',
             'Keong Siak',
             'Joo Chiat',
             'Serangoon Gardens Circus',
             'Orchard',
             'Arab Street',
             'Thomson Ridge',
             'Ang Siang Hill',
             'Amoy Street',
             'The Oval',
             'Sentosa'
             ]


In [3]:

results = {
    'address':[],
    'confidence':[],
    'latitude':[],
    'longitude':[],
    'quality':[]
}

for location in locations:
    g = geocoder.arcgis(location+' Singapore')
    results['address'].append(g.json['address'])
    results['confidence'].append(g.json['confidence'])
    results['latitude'].append(g.json['lat'])
    results['longitude'].append(g.json['lng'])
    results['quality'].append(g.json['quality'])

df = pd.DataFrame(results)
df

,address,confidence,latitude,longitude,quality
0,"Central Business District, Singapore",7,1.282720,103.851200,Locality
1,Holland Village,7,1.311940,103.793330,Locality
2,DEMPSEY HILL,7,1.304750,103.809800,POI
3,"Tiong Bahru, Singapore",7,1.289530,103.832080,Locality
4,"Rochester Park, 38 Rochester Park, 139242, Sin...",9,1.306102,103.786135,PointAddress
5,"Keong Saik Road, Singapore",9,1.280938,103.841753,StreetName
6,"Joo Chiat Lane, Singapore",9,1.312435,103.902417,StreetName
7,"Serangoon Garden Circus, Singapore",9,1.363996,103.865944,StreetName
8,"Orchard, Singapore",7,1.301090,103.839650,Locality
9,"Arab Street, Singapore",9,1.302327,103.858138,StreetName


In [4]:
import folium

locations = df[['latitude','longitude']]
locationlist = locations.values.tolist()

map = folium.Map(location=[1.3521, 103.8198])
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df['address'][point]).add_to(map)
map

In [5]:
poi_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude,df.latitude))
poi_df.crs = "EPSG:4326"

In [6]:
poi_df

,address,confidence,latitude,longitude,quality,geometry
0,"Central Business District, Singapore",7,1.282720,103.851200,Locality,POINT (103.85120 1.28272)
1,Holland Village,7,1.311940,103.793330,Locality,POINT (103.79333 1.31194)
2,DEMPSEY HILL,7,1.304750,103.809800,POI,POINT (103.80980 1.30475)
3,"Tiong Bahru, Singapore",7,1.289530,103.832080,Locality,POINT (103.83208 1.28953)
4,"Rochester Park, 38 Rochester Park, 139242, Sin...",9,1.306102,103.786135,PointAddress,POINT (103.78613 1.30610)
5,"Keong Saik Road, Singapore",9,1.280938,103.841753,StreetName,POINT (103.84175 1.28094)
6,"Joo Chiat Lane, Singapore",9,1.312435,103.902417,StreetName,POINT (103.90242 1.31243)
7,"Serangoon Garden Circus, Singapore",9,1.363996,103.865944,StreetName,POINT (103.86594 1.36400)
8,"Orchard, Singapore",7,1.301090,103.839650,Locality,POINT (103.83965 1.30109)
9,"Arab Street, Singapore",9,1.302327,103.858138,StreetName,POINT (103.85814 1.30233)


In [7]:
poi_df.to_postgis('points_of_interests',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})